# census-2021

In [1]:
# PACKAGES
import pandas as pd
from janitor import clean_names
import altair as alt
import requests

In [3]:
# DATA
url = 'https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/populationandhouseholdestimatesenglandandwalescensus2021/census2021/census2021firstresultsenglandwales1.xlsx'
filename = '../data/external/census2021firstresultsenglandwales1.xlsx'

In [ ]:
# FETCH
r = requests.get(url)
open(filename, 'wb').write(r.content)
df = pd.read_excel(filename)

In [120]:
# P01 Census 2021: Usual resident population by sex, local authorities in England and Wales
P01 = pd.read_excel(filename,
sheet_name="P01", header=6)

In [122]:
P01[P01['Area name'].isin(['Tower Hamlets', 'England', 'London'])]

,Area code [note 2],Area name,All persons,Females,Males
1,E92000001,England,56489800,28833500,27656300
211,E12000007,London,8799800,4531500,4268300
224,E09000030,Tower Hamlets,310300,154500,155800


In [117]:
# P02 Census 2021: Usual resident population by five-year age group, local authorities in England and Wales
P02 = pd.read_excel(filename,
sheet_name="P02", header=7)

In [118]:
P02[P02['Area name'].isin(['Tower Hamlets'])][['Aged 4 years and under\n[note 12]', 'Aged 5 to 9 years\n[note 12]','Aged 10 to 14 years\n[note 12]', 'Aged 15 to 19 years\n[note 12]','Aged 20 to 24 years\n[note 12]', 'Aged 25 to 29 years\n[note 12]','Aged 30 to 34 years\n[note 12]', 'Aged 35 to 39 years\n[note 12]','Aged 40 to 44 years\n[note 12]', 'Aged 45 to 49 years\n[note 12]','Aged 50 to 54 years\n[note 12]', 'Aged 55 to 59 years\n[note 12]','Aged 60 to 64 years\n[note 12]', 'Aged 65 to 69 years\n[note 12]','Aged 70 to 74 years\n[note 12]', 'Aged 75 to 79 years\n[note 12]','Aged 80 to 84 years\n[note 12]', 'Aged 85 to 89 years\n[note 12]','Aged 90 years and over\n[note 12]']]

,Aged 4 years and under\n[note 12],Aged 5 to 9 years\n[note 12],Aged 10 to 14 years\n[note 12],Aged 15 to 19 years\n[note 12],Aged 20 to 24 years\n[note 12],Aged 25 to 29 years\n[note 12],Aged 30 to 34 years\n[note 12],Aged 35 to 39 years\n[note 12],Aged 40 to 44 years\n[note 12],Aged 45 to 49 years\n[note 12],Aged 50 to 54 years\n[note 12],Aged 55 to 59 years\n[note 12],Aged 60 to 64 years\n[note 12],Aged 65 to 69 years\n[note 12],Aged 70 to 74 years\n[note 12],Aged 75 to 79 years\n[note 12],Aged 80 to 84 years\n[note 12],Aged 85 to 89 years\n[note 12],Aged 90 years and over\n[note 12]
224,19100,17700,17400,18400,32000,44500,40700,29800,22600,17500,13900,10800,8500,6000,4300,2800,2200,1300,700


In [4]:
# P03 Census 2021: Usual resident population by sex and five-year age group, local authorities in England and Wales
P03 = pd.read_excel(filename,
sheet_name="P03", header=7)

In [9]:
def reshape_gender_data(df):
    df.reset_index(drop=True, inplace=True)
    df = df.rename(columns=lambda x:x.replace('note 12',''))
    df = df.clean_names(remove_special=True)
    df = df.rename(columns=lambda x:x.replace('females_aged_',''))
    df = df.rename(columns=lambda x:x.replace('males_aged_',''))
    df = df.rename(columns=lambda x:x.replace('_years',''))
    df = df.rename(columns=lambda x:x.replace('_to_','-'))
    df = df.rename(columns=lambda x:x.replace('4_and_under','00-04'))
    df = df.rename(columns=lambda x:x.replace('_and_over','+'))
    df = df.rename(columns=lambda x:x.replace('5-9','05-09'))
    df = df.transpose()
    df.reset_index(inplace=True)
    df.columns = ['group','england and wales', 'london', 'Tower Hamlets']
    return df

In [10]:
P03_females = reshape_gender_data(P03[P03['Area name'].isin(['Tower Hamlets', 'England', 'London'])].filter(regex=r"Females.*"))
P03_males = reshape_gender_data(P03[P03['Area name'].isin(['Tower Hamlets', 'England', 'London'])].filter(regex=r"Males.*"))

In [13]:
def population_pyramid(females, males):

    females = alt.Chart(females).properties(width=250)
    males = alt.Chart(males).properties(width=250)

    left = (
        females.encode(
            y=alt.Y("group:O", axis=None),
            x=alt.X(
                "Tower Hamlets:Q",
                title="2021 population",
                sort=alt.SortOrder("descending"),
                axis=alt.Axis(tickCount=5),
            ),
            tooltip=["Tower Hamlets"],
        )
        .mark_bar()
        .properties(title="Tower Hamlets females")
    )

    middle = (
        females.encode(
            y=alt.Y("group:O", axis=None),
            text=alt.Text("group:O"),
        )
        .mark_text()
        .properties(width=20)
    )

    right = (
        males.encode(
            y=alt.Y("group:O", axis=None),
            x=alt.X(
                "Tower Hamlets:Q", title="2021 population", axis=alt.Axis(tickCount=5, )
            ),
            tooltip=["Tower Hamlets"],
        )
        .mark_bar()
        .properties(title="Tower Hamlets males")
    )

    return alt.concat(left, middle, right, spacing=5)


In [144]:
population_pyramid(P03_females, P03_males)

alt.ConcatChart(...)

In [28]:
# P04 Census 2021: Usual resident population density, local authorities in England and Wales
P04 = pd.read_excel(filename,
sheet_name="P04", header=6)

In [29]:
P04[P04['Area name'].isin(['Tower Hamlets', 'England', 'London'])]

,Area code [note 2],Area name,Population density (number of usual residents per square kilometre) \n[note 13]
1,E92000001,England,434
211,E12000007,London,5598
224,E09000030,Tower Hamlets,15695


In [31]:
# H01 Census 2021: Number of households with at least one usual resident, local authorities in England and Wales
H01 = pd.read_excel(filename,
sheet_name="H01", header=6)

In [32]:
H01[H01['Area name'].isin(['Tower Hamlets', 'England', 'London'])]

,Area code [note 2],Area name,Number of households with at least one usual resident
1,E92000001,England,23435700
211,E12000007,London,3423800
224,E09000030,Tower Hamlets,120500


In [2]:
# create a html output
! jupyter nbconvert --to html 1.0-uk-census-2021-eda.ipynb --no-input

[NbConvertApp] Converting notebook 1.0-uk-census-2021-eda.ipynb to html
[NbConvertApp] Writing 588995 bytes to 1.0-uk-census-2021-eda.html


: 